In [ ]:
### 발산역 근처 top5 아파트 시세
# 우장산 힐스테이트, 우장산 월드메르디앙, 등촌주공 3단지, 우장산 롯데캐슬, 마곡수명산파크 1단지

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

In [3]:
## 복호화함수
import base64
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad



def decrypt(encrypted_text, secret):
    """
    AES ECB 모드로 암호화된 텍스트를 복호화합니다.
    
    Args:
        encrypted_text (str): Base64로 인코딩된 암호화된 텍스트
    
    Returns:
        str: 복호화된 평문 텍스트
    """
    # Base64 디코딩
    encrypted_bytes = base64.b64decode(encrypted_text)
    
    # 키 생성
    key = get_key(secret)
    
    # AES 복호화 객체 생성 (ECB 모드)
    cipher = AES.new(key, AES.MODE_ECB)
    
    # 복호화
    decrypted_bytes = cipher.decrypt(encrypted_bytes)
    
    # PKCS7 패딩 제거
    unpadded_bytes = unpad(decrypted_bytes, AES.block_size)
    
    # UTF-8 문자열로 변환
    return unpadded_bytes.decode('utf-8')

def get_key(secret):
    """
    비밀 문자열을 AES 키로 변환합니다.
    키 길이에 따라 적절한 크기로 조정합니다.
    
    Args:
        secret (str): 비밀 문자열
    
    Returns:
        bytes: AES 키 (16, 24, 또는 32 바이트)
    """
    # 문자열을 UTF-8 바이트로 변환
    key_bytes = secret.encode('utf-8')
    length = len(key_bytes)
    
    # 키 길이에 따라 조정
    if length < 16:
        # 16바이트보다 작으면 16바이트로 패딩
        adjusted_key = key_bytes[:16].ljust(16, b'\x00')
    elif length > 16 and length < 24:
        # 16바이트보다 크고 24바이트보다 작으면 24바이트로 조정
        adjusted_key = key_bytes[:24].ljust(24, b'\x00')
    elif length > 24 and length < 32:
        # 24바이트보다 크고 32바이트보다 작으면 32바이트로 조정
        adjusted_key = key_bytes[:32].ljust(32, b'\x00')
    elif length >= 32:
        # 32바이트 이상이면 32바이트로 잘라내기
        adjusted_key = key_bytes[:32]
    else:
        # 정확히 16바이트면 그대로 사용
        adjusted_key = key_bytes
    
    return adjusted_key

In [4]:
# 억단위 숫자로
import re

def parse_korean_price(text):
    """
    '1억 2,300' → 123000000 변환
    """
    if pd.isna(text):
        return None
    text = text.replace(',', '').replace(' ', '')
    total = 0
    match = re.match(r'(?:(\d+)억)?(?:(\d+)천)?(\d+)?', text)
    if not match:
        return None
    if match.group(1):  # 억
        total += int(match.group(1)) * 100_000_000
    if match.group(2):  # 천만 단위
        total += int(match.group(2)) * 10_000_000
    if match.group(3):  # 만 단위 이하
        total += int(match.group(3))
    return total

In [ ]:
## 우장산 힐스테이트
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=448399"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['759369526619114448399']


In [6]:
## 우장산 힐스테이트
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=448399&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_1 = pd.DataFrame(rows)

key = '759369526619114448399'

# 가격과 전월세 컬럼 복호화
df_1['가격'] = df_1['가격'].apply(lambda x: decrypt(x, key))
df_1['전월세'] = df_1['전월세'].apply(lambda x: decrypt(x, key))

df_1['가격(숫자)'] = df_1['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_1

,거래일,층수,가격,전월세,가격(숫자)
0,2025.06.27,6,"11억 9,800",,1100009800
1,2025.06.27,6,"11억 9,500",,1100009500
2,2025.06.23,4,"13억 6,000",,1300006000
3,2025.06.22,14,"12억 2,000",,1200002000
4,2025.06.18,15,"12억 4,000",,1200004000
...,...,...,...,...,...
995,2014.10.21,21,"5억 7,000",,500007000
996,2014.10.18,21,"5억 6,600",,500006600
997,2014.10.15,6,"5억 9,000",,500009000
998,2014.10.14,12,"5억 6,900",,500006900


In [7]:
## 우장산 월드메르디앙
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=20008575"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['75536880289260220008575']


In [9]:
## 우장산 월드메리드앙
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=20008575&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_2 = pd.DataFrame(rows)

key = '75536880289260220008575'

# 가격과 전월세 컬럼 복호화
df_2['가격'] = df_2['가격'].apply(lambda x: decrypt(x, key))
df_2['전월세'] = df_2['전월세'].apply(lambda x: decrypt(x, key))

df_2['가격(숫자)'] = df_2['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_2

,거래일,층수,가격,전월세,가격(숫자)
0,2025.05.26,5,"8억 3,750",,800003750
1,2025.05.18,14,8억,,800000000
2,2025.03.06,4,"8억 3,000",,800003000
3,2025.01.22,12,"8억 5,500",,800005500
4,2024.09.13,11,"8억 4,000",,800004000
...,...,...,...,...,...
93,2006.11.01,11,"3억 5,000",,300005000
94,2006.10.28,13,"3억 6,500",,300006500
95,2006.10.01,9,"3억 7,500",,300007500
96,2006.09.23,14,"3억 8,200",,300008200


In [ ]:
## 등촌주공3단지
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=1797"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['7573695431433881797']


In [11]:
## 등촌주공 3단지
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=1797&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_3 = pd.DataFrame(rows)

key = '7573695431433881797'

# 가격과 전월세 컬럼 복호화
df_3['가격'] = df_3['가격'].apply(lambda x: decrypt(x, key))
df_3['전월세'] = df_3['전월세'].apply(lambda x: decrypt(x, key))

df_3['가격(숫자)'] = df_3['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_3

,거래일,층수,가격,전월세,가격(숫자)
0,2025.07.04,2,"8억 6,500",,800006500
1,2025.07.04,2,"8억 6,500",,800006500
2,2025.07.01,7,"6억 2,500",,600002500
3,2025.06.28,13,"8억 9,000",,800009000
4,2025.06.27,4,"8억 6,000",,800006000
...,...,...,...,...,...
995,2007.08.21,10,"1억 8,500",,100008500
996,2007.08.20,4,"1억 7,380",,100007380
997,2007.07.29,14,"3억 2,000",,300002000
998,2007.06.30,9,3억,,300000000


In [12]:
## 우장산롯데캐슬
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=50602"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['75236954133760350602']


In [13]:
## 우장산 롯데캐슬
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=50602&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_4 = pd.DataFrame(rows)

key = '75236954133760350602'

# 가격과 전월세 컬럼 복호화
df_4['가격'] = df_4['가격'].apply(lambda x: decrypt(x, key))
df_4['전월세'] = df_4['전월세'].apply(lambda x: decrypt(x, key))

df_4['가격(숫자)'] = df_4['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_4

,거래일,층수,가격,전월세,가격(숫자)
0,2025.07.03,22,"11억 5,000",,1100005000
1,2025.06.23,16,13억,,1300000000
2,2025.06.19,3,"13억 5,000",,1300005000
3,2025.06.19,12,"11억 8,000",,1100008000
4,2025.06.17,23,"11억 5,000",,1100005000
...,...,...,...,...,...
982,2006.02.02,14,"4억 2,900",,400002900
983,2006.02.02,18,"4억 3,000",,400003000
984,2006.01.28,11,"4억 3,800",,400003800
985,2006.01.26,15,"4억 3,700",,400003700


In [14]:
## 마곡수명산파크1단지
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=20073246"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['75636880049977520073246']


In [15]:
## 마곡수명산파크1단지
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=20073246&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_5 = pd.DataFrame(rows)

key = '75636880049977520073246'

# 가격과 전월세 컬럼 복호화
df_5['가격'] = df_5['가격'].apply(lambda x: decrypt(x, key))
df_5['전월세'] = df_5['전월세'].apply(lambda x: decrypt(x, key))

df_5['가격(숫자)'] = df_5['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_5

,거래일,층수,가격,전월세,가격(숫자)
0,2025.07.19,7,"10억 9,000",,1000009000
1,2025.06.26,9,"11억 3,000",,1100003000
2,2025.06.21,9,"10억 9,000",,1000009000
3,2025.06.21,9,"10억 9,000",,1000009000
4,2025.06.20,8,"11억 6,000",,1100006000
...,...,...,...,...,...
723,2007.11.21,5,"2억 5,575",,200005575
724,2007.11.13,2,"2억 3,093",,200003093
725,2007.11.08,5,"2억 3,533",,200003533
726,2007.11.06,5,"2억 3,614",,200003614
